# Procesamiento de Libros y Entrenamiento de Modelos Word2Vec

Este notebook procesa una colección de libros de tres autores (i.e., Jane Austen, Leo Tolstoy y James Joyce), aplicando preprocesamiento de texto y entrenando modelos Word2Vec con diferentes dimensiones vectoriales para capturar relaciones semánticas entre las palabras.

## 0. Importación de Librerías

In [1]:
import pickle
import re

import gensim
from gensim.models import Word2Vec
from gensim.parsing.preprocessing import STOPWORDS
from gensim.utils import simple_preprocess

## 1. Definición de Funciones

In [2]:
def preprocess_text_gensim(text):
    """
    Limpia y tokeniza el texto mediante:
    1. Eliminación de puntuación y caracteres especiales.
    2. Convierte el texto a minúsculas.
    3. Tokenización del texto en palabras.
    4. Eliminación de palabras vacías (stopwords).
    
    Args:
    text (str): Texto de entrada a preprocesar.
    
    Returns:
    list: Una lista de tokens (palabras) del texto limpiado.
    """
    # Eliminar cualquier carácter no alfabético, números, etc.
    text = re.sub(r'[^\w\s]', '', text)
    
    # Tokenizar y convertir el texto a minúsculas
    tokens = gensim.utils.simple_preprocess(text, deacc=True)
    
    # Eliminar palabras vacías (stopwords)
    tokens = [word for word in tokens if word not in STOPWORDS]
    
    return tokens

def prepare_corpus_gensim(text):
    """
    Divide el texto en oraciones y aplica el preprocesamiento a cada oración.
    
    Args:
    text (str): Texto de entrada para dividir y preprocesar.
    
    Returns:
    lista de listas: Oraciones tokenizadas, donde cada oración es una lista de tokens.
    """
    # Dividir el texto en oraciones (dividiendo por puntos o saltos de línea)
    sentences = text.split('.')    # Alternativamente, usar '\n' para párrafos u otros delimitadores
    
    # Preprocesar cada oración usando el preprocesamiento de Gensim
    tokenized_sentences = [preprocess_text_gensim(sentence) for sentence in sentences if sentence]
    
    return tokenized_sentences

def process_single_book(file_path, output_path):
    """
    Lee un archivo de libro, preprocesa el texto y guarda el corpus tokenizado.
    
    Args:
    file_path (str): Ruta al archivo del libro sin procesar.
    output_path (str): Ruta para guardar los datos preprocesados.
    """
    # Leer el texto del libro
    with open(file_path, 'r', encoding='utf-8') as f:
        text = f.read()
    
    # Preprocesar y preparar el corpus
    corpus = prepare_corpus_gensim(text)
    
    # Guardar el corpus tokenizado en un archivo para uso futuro
    with open(output_path, 'wb') as f:
        pickle.dump(corpus, f)

def load_processed_data(file_path):
    """
    Carga el corpus tokenizado preprocesado desde un archivo pickle.
    
    Args:
    file_path (str): Ruta a los datos preprocesados.
    
    Returns:
    list of lists: El corpus tokenizado.
    """
    with open(file_path, 'rb') as f:
        corpus = pickle.load(f)
    return corpus

def train_word2vec(corpus, vector_size, window, min_count, epochs, save_path):
    """
    Entrena un modelo Word2Vec usando Gensim y guarda el modelo entrenado en un archivo.
    
    Args:
    corpus (list of lists): El corpus tokenizado.
    vector_size (int): Dimensión de los vectores de palabras.
    window (int): Tamaño de la ventana de contexto.
    min_count (int): Número mínimo de veces que una palabra debe aparecer para ser incluida.
    epochs (int): Número de épocas de entrenamiento.
    save_path (str): Ruta para guardar el modelo entrenado.
    
    Returns:
    gensim.models.Word2Vec: Modelo Word2Vec entrenado.
    """
    # Entrenar el modelo Word2Vec
    model = Word2Vec(sentences=corpus, vector_size=vector_size, window=window, min_count=min_count, sg=1, epochs=epochs)
    
    # Guardar el modelo entrenado en un archivo
    model.save(save_path)
    
    return model

## 2. Procesamiento de los Libros

In [3]:
# Ruta a los libros originales
raw_books = {
    'austen_sense-and-sensibility': {
        'file_path': 'data/raw/austen_sense-and-sensibility.txt',
        'output_path': 'data/processed/austen_sense-and-sensibility.pkl'
    },
    'austen_pride-and-prejudice': {
        'file_path': 'data/raw/austen_pride-and-prejudice.txt',
        'output_path': 'data/processed/austen_pride-and-prejudice.pkl'
    },
    'austen_emma': {
        'file_path': 'data/raw/austen_emma.txt',
        'output_path': 'data/processed/austen_emma.pkl'
    },
    'tolstoy_youth': {
        'file_path': 'data/raw/tolstoy_youth.txt',
        'output_path': 'data/processed/tolstoy_youth.pkl'
    },
    'tolstoy_war-and-peace': {
        'file_path': 'data/raw/tolstoy_war-and-peace.txt',
        'output_path': 'data/processed/tolstoy_war-and-peace.pkl'
    },
    'tolstoy_anna-karenina': {
        'file_path': 'data/raw/tolstoy_anna-karenina.txt',
        'output_path': 'data/processed/tolstoy_anna-karenina.pkl'
    },
    'joyce_dubliners': {
        'file_path': 'data/raw/joyce_dubliners.txt',
        'output_path': 'data/processed/joyce_dubliners.pkl'
    },
    'joyce_a-portrait-of-the-artist-as-a-young-man': {
        'file_path': 'data/raw/joyce_a-portrait-of-the-artist-as-a-young-man.txt',
        'output_path': 'data/processed/joyce_a-portrait-of-the-artist-as-a-young-man.pkl'
    },
    'joyce_ulysses': {
        'file_path': 'data/raw/joyce_ulysses.txt',
        'output_path': 'data/processed/joyce_ulysses.pkl'
    }
}

# Procesar cada libro
for book, paths in raw_books.items():
    print(f"Procesando {book}...")
    process_single_book(paths['file_path'], paths['output_path'])

Procesando austen_sense-and-sensibility...
Procesando austen_pride-and-prejudice...
Procesando austen_emma...
Procesando tolstoy_youth...
Procesando tolstoy_war-and-peace...
Procesando tolstoy_anna-karenina...
Procesando joyce_dubliners...
Procesando joyce_a-portrait-of-the-artist-as-a-young-man...
Procesando joyce_ulysses...


## 3. Entrenamiento de los Modelos Word2Vec

In [4]:
# Ruta a los libros procesados
processed_books = {
    'austen_sense-and-sensibility': 'data/processed/austen_sense-and-sensibility.pkl',
    'austen_pride-and-prejudice': 'data/processed/austen_pride-and-prejudice.pkl',
    'austen_emma': 'data/processed/austen_emma.pkl',
    'tolstoy_youth': 'data/processed/tolstoy_youth.pkl',
    'tolstoy_war-and-peace': 'data/processed/tolstoy_war-and-peace.pkl',
    'tolstoy_anna-karenina': 'data/processed/tolstoy_anna-karenina.pkl',
    'joyce_dubliners': 'data/processed/joyce_dubliners.pkl',
    'joyce_a-portrait-of-the-artist-as-a-young-man': 'data/processed/joyce_a-portrait-of-the-artist-as-a-young-man.pkl',
    'joyce_ulysses': 'data/processed/joyce_ulysses.pkl'
}

# Código de nuestro grupo
group_code = 'CarlosRaulDeLaRosaPeredoJhonStewarRayoMosqueraMarioGarridoCordoba'

# Cargar y combinar todos los corpus de los libros
combined_corpus = []
for file_path in processed_books.values():
    corpus = load_processed_data(file_path)
    combined_corpus.extend(corpus)    # Añadir cada corpus al corpus combinado

# Entrenamiento del modelo combinado con diferentes tamaños de vectores
vector_sizes = [50, 100, 300]  # Dimensiones de los embeddings a probar
window_size = 5  # Tamaño de la ventana de contexto
min_count = 3    # Solo considerar palabras que aparezcan al menos 3 veces
epochs = 10      # Número de épocas de entrenamiento

# Entrenar los modelos Word2Vec de 50, 100 y 300 dimensiones de todos los libros
for vector_size in vector_sizes:
    print(f"Entrenando modelo combinado con {vector_size} dimensiones...")
    
    # Guardar el modelo combinado con el tamaño de vector especificado
    model_save_path = f"data/models/Books_{vector_size}_{group_code}.model"
    
    # Entrenar el modelo con el tamaño de vector especificado
    model = train_word2vec(combined_corpus, vector_size=vector_size, window=window_size, min_count=min_count, epochs=epochs, save_path=model_save_path)

Entrenando modelo combinado con 50 dimensiones...
Entrenando modelo combinado con 100 dimensiones...
Entrenando modelo combinado con 300 dimensiones...
